In [ ]:
from langchain_community.chat_models import BedrockChat
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv
import os
import boto3
import json


In [ ]:
# .envファイルからAWSのプロファイルを取得
load_dotenv()


In [ ]:
chat = BedrockChat(model_id="anthropic.claude-v2:1", model_kwargs={"temperature": 0})
# chat = BedrockChat(model_id="anthropic.claude-3-haiku-20240307-v1:0", model_kwargs={"temperature": 0.1}, region_name="us-east-1") # 3は対応できていない?


In [ ]:
chat.invoke([HumanMessage('こんにちは、日本語の慣用表現についての4択問題を出してください')])


In [ ]:
client = boto3.client(service_name='bedrock-runtime', region_name='us-east-1')


In [ ]:
jenre = '日本史'

# prompt = 'こんにちは、日本語の慣用表現についての4択問題を出してください'
prompt = f'''\
### 指示
あなたは有名なクイズ作家兼プログラマーです。与えられたジャンルに関する4択クイズを考えて、指定した形式で出力してください。

### 条件
・出力内容としては「ジャンル」、「問題」、「選択肢1」、「選択肢2」、「選択肢3」、「選択肢4」、「回答番号」、「解説」にしてください。
・「回答番号」は「選択肢n」のように「選択肢」と番号の文字列をつなげたものにしてください。
・クイズはWebサイトで出題、回答させることを想定している為、プログラムで扱いやすいJSON形式で返してください。
・「回答番号」については偏りがあると回答者が真剣に考えることなく選択する可能性があるため、適度にランダムに選択してください。

### 出力(JSON)例
{{
    "ジャンル": "ことわざ",
    "問題": "「猿も木から落ちる」の意味は何ですか？",
    "選択肢1": "猿は木から降りるのが下手",
    "選択肢2": "熟練した人でも失敗することがある",
    "選択肢3": "木が高ければ高いほど落ちやすい",
    "選択肢4": "猿は木を壊す",
    "回答番号": "選択肢2",
    "解説": "このことわざは、普段猿が木登りが上手であるにも関わらず、時には木から落ちることがあるという事実から来ています。\\nそれに喩えて、どんなに技術が高く、経験が豊富な人であっても、時には間違いや失敗を犯すことがあるという意味を込めています。"
}}
    
### ジャンル
{jenre}

### 出力(JSON)
'''

print(prompt)


In [ ]:
# model_id = 'anthropic.claude-3-sonnet-20240229-v1:0'
model_id = 'anthropic.claude-3-haiku-20240307-v1:0'

response = client.invoke_model(
                modelId=model_id,
                body=json.dumps(
                    {
                        "anthropic_version": "bedrock-2023-05-31",
                        "max_tokens": 1024,
                        "messages": [
                            {
                                "role": "user",
                                "content": [{"type": "text", "text": prompt}],
                            }
                        ],
                    }
                ),
            )


In [ ]:
result = json.loads(response.get('body').read())


In [ ]:
input_tokens = result["usage"]["input_tokens"]
output_tokens = result["usage"]["output_tokens"]
output_list = result.get("content", [])

print("Invocation details:")
print(f"- The input length is {input_tokens} tokens.")
print(f"- The output length is {output_tokens} tokens.")

print(f"- The model returned {len(output_list)} response(s):")
for output in output_list:
    print(output["text"])
    

In [ ]:
print(output_list[0]['text'])
